In [1]:
import pyodbc
import pandas as pd
import os
from IPython.display import display, Markdown

dbconn = os.environ.get('DBCONN', None)
if dbconn is None:
    display("No SQL credentials. Check that the file 'environ.txt' is present. Refer to readme for further information")
else:
    dbconn = dbconn.strip('"')

# import custom functions from 'analysis' folder
import sys
sys.path.append('../lib/')
    
from functions import plotting_all, closing_connection

## Inspect Top-Level CTV3 Codes

In [2]:
sql = '''select 
LEFT(CTV3Code,1) AS first_digit,
'1' AS digits,
Description
from CTV3Dictionary
where RIGHT(CTV3Code,4)='....' 
and LEFT(CTV3Code,1) NOT IN (
'.', -- Read thesaurus
'0', -- occupations
'9') -- Administration
'''

pd.set_option('display.max_rows', 200)
with closing_connection(dbconn) as connection:
    codes = pd.read_sql(sql, connection)
codes

,first_digit,digits,Description
0,1,1,History/symptoms
1,2,1,(Examination of patient) or (physical signs)
2,3,1,Diagnostic procedures
3,4,1,Laboratory procedures
4,5,1,(Radiol &/or physics in medicine) or (radiolog...
5,6,1,Preventive procedure
6,7,1,(Surgical operations) or (sites)
7,8,1,Other therapeutic procedures
8,A,1,Infective disorder
9,B,1,(Neoplasms) or (cancers)


## Inspect Level 2 CTV3 Codes

In [3]:
digit = "1"

sql = f'''select 
LEFT(CTV3Code,2) AS first_digit,
'2' AS digits,
Description
from CTV3Dictionary
where RIGHT(CTV3Code,3)='...' 
AND RIGHT(CTV3Code,4)!='....' 
and CAST(LEFT(CTV3Code,1) AS VARCHAR) = '{digit}'
'''

pd.set_option('display.max_rows', 200)
with closing_connection(dbconn) as connection:
    codes = pd.read_sql(sql, connection)
codes


,first_digit,digits,Description
0,11,2,Depth of history
1,12,2,Family history
2,13,2,Social and personal history
3,14,2,Past medical history
4,15,2,[Gynaecological] or [obstetric] history
5,16,2,[General] or [vague] or [multiple] symptoms
6,17,2,Respiratory symptoms
7,18,2,Cardiovascular symptoms (& [heart])
8,19,2,Gastrointestinal symptoms (& [intestinal])
9,1A,2,Symptoms: [genitourinary] or [urinary]


## Create table of level 1 and level 2 CTV3 codes

In [4]:
digit = "2"

sql = f'''select 
LEFT(CTV3Code,CHARINDEX('.',CTV3Code)-1) AS first_digit,
CHARINDEX('.',CTV3Code)-1 AS digits,
Description
from CTV3Dictionary
where (RIGHT(CTV3Code,3)='...' OR RIGHT(CTV3Code,4)='....')
and LEFT(CTV3Code,1) NOT IN (
'.', -- Read thesaurus
'0', -- occupations
'1', -- History/symptoms
'9') -- Administration
'''

pd.set_option('display.max_rows', 200)
with closing_connection(dbconn) as connection:
    codes = pd.read_sql(sql, connection)
codes

,first_digit,digits,Description
0,2,1,(Examination of patient) or (physical signs)
1,21,2,Depth of examination
2,22,2,General examination of patient
3,23,2,(Examination of respirat system) or (O/E - che...
4,24,2,Examination of cardiovascular system (& [vascu...
...,...,...,...
849,u,1,Contrast media
850,u1,2,X-ray contrast media
851,u2,2,Magnetic resonance imaging contrast media
852,u3,2,Ultrasound contrast media


## Create table of level 2 & 3 CTV3 codes

In [5]:
digit = "4"

sql = f'''select 
LEFT(CTV3Code,CHARINDEX('.',CTV3Code)-1) AS first_digit,
CHARINDEX('.',CTV3Code)-1 AS digits,
Description
from CTV3Dictionary
where RIGHT(CTV3Code,2)='..' --(select codes like '123..')
and RIGHT(CTV3Code,4)!='....' --(don't select codes like '1....')
and LEFT(CTV3Code,1) NOT IN ('.') -- Read thesaurus
and LEFT(CTV3Code,1) = '{digit}'
'''

pd.set_option('display.max_rows', 400)
with closing_connection(dbconn) as connection:
    codes_l3 = pd.read_sql(sql, connection)

codes_l3

,first_digit,digits,Description
0,41,2,"(Gen lab proc (& [invest][test])) or (result, ..."
1,411,3,Laboratory test not necessary
2,412,3,Laboratory procedure performed
3,413,3,Laboratory test requested
4,414,3,Sample sent to laboratory for test
5,415,3,Patient refused laboratory test
6,416,3,Laboratory test requested - not done
7,417,3,(Laboratory test result not back yet) or (awai...
8,418,3,Lab. test result normal
9,419,3,Laboratory test result abnormal


## Rank level 2 codes by frequency of appearance in 2020

In [6]:
highlevelcode='4'
subset = codes.loc[codes["first_digit"].str[0]== highlevelcode]

codelist = tuple(subset.first_digit)

# select events for selcted subset of codelist
sql = f'''select LEFT(CTV3Code,2) AS first_digit, COUNT(Patient_ID) as events
FROM CodedEvent 
WHERE 
ConsultationDate IS NOT NULL 
AND CAST(LEFT(CTV3Code,2) AS VARCHAR) IN {codelist}
AND YEAR(ConsultationDate) = 2020
GROUP BY LEFT(CTV3Code,2)
'''
with closing_connection(dbconn) as connection:
    df = pd.read_sql(sql, connection).sort_values(by="events", ascending=False)
top_l2 = df.merge(subset, on="first_digit", how="left")

top_l2 = top_l2.loc[top_l2["events"]>100000]
top_l2["events (millions)"] = round(top_l2["events"]/1000000, 2)
top_l2 = top_l2.drop("events", axis=1)
top_l2.head(30)


,first_digit,digits,Description,events (millions)


## Rank level 3 codes by frequency of appearance in 2020

In [8]:
codelist = tuple(codes_l3.first_digit)

# select events for selcted subset of codelist
sql = f'''select TOP 50 
LEFT(CTV3Code,3) AS first_digit, COUNT(Patient_ID) as events
FROM CodedEvent 
WHERE 
ConsultationDate IS NOT NULL 
AND CAST(LEFT(CTV3Code,3) AS VARCHAR) IN {codelist}
AND YEAR(ConsultationDate) = 2020
GROUP BY LEFT(CTV3Code,3)
ORDER BY events DESC
'''

with closing_connection(dbconn) as connection:
    df = pd.read_sql(sql, connection).sort_values(by="events", ascending=False)
top_l3 = df.merge(codes_l3, on="first_digit", how="left")

top_l3 = top_l3.loc[top_l3["events"]>10000]
top_l3["events (millions)"] = round(top_l3["events"]/1000000, 2)
top_l3 = top_l3.drop("events", axis=1)
top_l3.head(30)

,first_digit,digits,Description,events (millions)
